# 

# 3_Jsonfy and preprocess to mmap format for optimizing data loading
---

## Learning Objectives
- **The goal of this lab is to re-use the knowledge we gain from Day 2 and utilize that knowledge to jsonfy our cleaned text data and convert to mmap format**
    -  jsonfy the extract webnyheter2013.txt --> webnyheter2013.json
    -  mini-challenge - 
        - task 1 : integrate a customized sentence splitter into the preprocess_data.py, let's call it MYpreprocess_data.py  
                        
                        ---------the customized sentence splitter is this function below ------------
        
                            import re
                            import nltk
                            from nltk.tokenize import sent_tokenize
                            def normal_cut_sentence(temp):
                                return sent_tokenize(temp)

                            def cut_sentence_with_quotation_marks(text):
                                p = re.compile("“.*?”")
                                list = []
                                index = 0
                                length = len(text)
                                for i in p.finditer(text):
                                    temp = ''
                                    start = i.start()
                                    end = i.end()
                                    for j in range(index, start):
                                        temp += text[j]
                                    if temp != '':
                                        temp_list = normal_cut_sentence(temp)
                                        list += temp_list
                                    temp = ''
                                    for k in range(start, end):
                                        temp += text[k]
                                    if temp != ' ':
                                        list.append(temp)
                                    index = end
                                return list
        
        task 2 : succesfully run the new MYpreprocess_data.py on example dataset **./Megatron-LM/dataset/EN/extractedNVblogs.json** 

            hint : open ./Megatron-LM/tools/MYpreprocess_data.py and modify the class between L55 -L91 in 
             
        ![modify the class below](./Megatron-LM/pics/customize_preprocess_data_script.JPG)
            


---
## jsonfy the extract webnyheter2013.txt --> webnyheter2013.json

In [1]:
!python create_loose_json.py --infile ../dataset/SV/webnyheter2013.txt --outfile ../dataset/SV/webnyheter2013.json

process 1000000 documents so far ...
example:  – Vi har en bra generation som spelat tillsammans ett tag .

finished processing 1249010 lines to loose json format


---
# run through the default preprocess_data.py to obtain the xxx.idx and xxx.bin files

In [7]:
INPUT_JSON_FILE='../dataset/SV/webnyheter2013.json'
OUTPUT_PATH='../dataset/SV/webnyheter2013_56kvocab'
VOCAB_FILE='../dataset/SV/56k/vocab.json'
MERGE_FILE='../dataset/SV/56k/merges.txt'
NUM_CPUS=16

In [8]:
!python ./Megatron-LM/tools/preprocess_data.py \
                       --input $INPUT_JSON_FILE \
                       --output-prefix $OUTPUT_PATH \
                       --json-keys text \
                       --vocab-file $VOCAB_FILE \
                       --merge-file $MERGE_FILE \
                       --dataset-impl mmap \
                       --tokenizer-type GPT2BPETokenizer \
                       --workers $NUM_CPUS \
                       --append-eod

Opening ../dataset/SV/webnyheter2013.json
> building GPT2BPETokenizer tokenizer ...
 > padded vocab (size: 56000) with 64 dummy tokens (new size: 56064)
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
Vocab size: 56000
Output prefix: ../dataset/SV/webnyheter2013_56kvocab
Time to startup: 0.17174911499023438
 > padded vocab (size: 56000) with 64 dummy tokens (new size: 56064

---
## cp the preprocess_data.py into a new python script called MYpreprocess_data.py

In [2]:
!cp ./Megatron-LM/tools/preprocess_data.py ./Megatron-LM/tools/MYpreprocess_data.py

In [3]:
import re
import nltk
from nltk.tokenize import sent_tokenize
def normal_cut_sentence(temp):
    return sent_tokenize(temp)

def cut_sentence_with_quotation_marks(text):
    p = re.compile("“.*?”")
    list = []
    index = 0
    length = len(text)
    for i in p.finditer(text):
        temp = ''
        start = i.start()
        end = i.end()
        for j in range(index, start):
            temp += text[j]
        if temp != '':
            temp_list = normal_cut_sentence(temp)
            list += temp_list
        temp = ''
        for k in range(start, end):
            temp += text[k]
        if temp != ' ':
            list.append(temp)
        index = end
    return list

----------------

## **Challenge ** - customize your own MYpreprocess_data.py

Task : modify the below script and overwrite MYpreprocess_data.py

Modify MYpreprocess_data.py below to incoporate the custom **cut_sentence_with_quotation_marks**

<a id="MODIFY_CELL"></a>

<a href="./Day3-4_customize_process2mmap.ipynb#Rerun_Cell">Jump to ReRun Cell</a> 

In [2]:
%%writefile ./Megatron-LM/tools/MYpreprocess_data.py 
# coding=utf-8
# Copyright (c) 2020, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Processing data for pretraining."""

import argparse
import json
import multiprocessing
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__),
                                             os.path.pardir)))
import time

import torch
try:
    import nltk
    nltk_available = True
except ImportError:
    nltk_available = False

from megatron.tokenizer import build_tokenizer
from megatron.data import indexed_dataset


# https://stackoverflow.com/questions/33139531/preserve-empty-lines-with-nltks-punkt-tokenizer
class CustomLanguageVars(nltk.tokenize.punkt.PunktLanguageVars):

    _period_context_fmt = r"""
        \S*                          # some word material
        %(SentEndChars)s             # a potential sentence ending
        \s*                       #  <-- THIS is what I changed
        (?=(?P<after_tok>
            %(NonWord)s              # either other punctuation
            |
            (?P<next_tok>\S+)     #  <-- Normally you would have \s+ here
        ))"""

class IdentitySplitter(object):
    def tokenize(self, *text):
        return text
"""[TODO]: modify this class to integrate the custom sentence splitter above """

class Encoder(object):
    def __init__(self, args):
        self.args = args
    
    def initializer(self):
        # Use Encoder class as a container for global data
        Encoder.tokenizer = build_tokenizer(self.args)
        if self.args.split_sentences:
            if not nltk_available:
                print("NLTK is not available to split sentences.")
                exit()
            splitter = nltk.load("tokenizers/punkt/english.pickle")
            if self.args.keep_newlines:
                # this prevents punkt from eating newlines after sentences
                Encoder.splitter = nltk.tokenize.punkt.PunktSentenceTokenizer(
                    train_text = splitter._params,
                    lang_vars = CustomLanguageVars())
            else:
                Encoder.splitter = splitter

        else:
            Encoder.splitter = IdentitySplitter()

    def encode(self, json_line):
        data = json.loads(json_line)
        ids = {}
        for key in self.args.json_keys:
            text = data[key]
            doc_ids = []
            for sentence in Encoder.splitter.tokenize(text):
                sentence_ids = Encoder.tokenizer.tokenize(sentence)
                if len(sentence_ids) > 0:
                    doc_ids.append(sentence_ids)
            if len(doc_ids) > 0 and self.args.append_eod:
                doc_ids[-1].append(Encoder.tokenizer.eod)
            ids[key] = doc_ids
        return ids, len(json_line)

def get_args():
    parser = argparse.ArgumentParser()
    group = parser.add_argument_group(title='input data')
    group.add_argument('--input', type=str, required=True,
                       help='Path to input JSON')
    group.add_argument('--json-keys', nargs='+', default=['text'],
                       help='space separate listed of keys to extract from json')
    group.add_argument('--split-sentences', action='store_true',
                       help='Split documents into sentences.')
    group.add_argument('--keep-newlines', action='store_true',
                       help='Keep newlines between sentences when splitting.')

    group = parser.add_argument_group(title='tokenizer')
    group.add_argument('--tokenizer-type', type=str, required=True,
                       choices=['BertWordPieceLowerCase','BertWordPieceCase',
                                'GPT2BPETokenizer'],
                       help='What type of tokenizer to use.')
    group.add_argument('--vocab-file', type=str, default=None,
                       help='Path to the vocab file')
    group.add_argument('--merge-file', type=str, default=None,
                       help='Path to the BPE merge file (if necessary).')
    group.add_argument('--append-eod', action='store_true',
                       help='Append an <eod> token to the end of a document.')


    group = parser.add_argument_group(title='output data')
    group.add_argument('--output-prefix', type=str, required=True,
                       help='Path to binary output file without suffix')
    group.add_argument('--dataset-impl', type=str, default='mmap',
                       choices=['lazy', 'cached', 'mmap'])

    group = parser.add_argument_group(title='runtime')
    group.add_argument('--workers', type=int, default=1,
                       help='Number of worker processes to launch')
    group.add_argument('--log-interval', type=int, default=100,
                       help='Interval between progress updates')
    args = parser.parse_args()
    args.keep_empty = False

    if args.tokenizer_type.lower().startswith('bert'):
        if not args.split_sentences:
            print("Bert tokenizer detected, are you sure you don't want to split sentences?")

    # some default/dummy values for the tokenizer
    args.rank = 0
    args.make_vocab_size_divisible_by = 128
    args.tensor_model_parallel_size = 1
    args.vocab_extra_ids = 0

    return args

def main():
    args = get_args()
    startup_start = time.time()

    print("Opening", args.input)
    fin = open(args.input, 'r', encoding='utf-8')

    if nltk_available and args.split_sentences:
        nltk.download("punkt", quiet=True)

    encoder = Encoder(args)
    tokenizer = build_tokenizer(args)
    pool = multiprocessing.Pool(args.workers, initializer=encoder.initializer)
    encoded_docs = pool.imap(encoder.encode, fin, 25)
    #encoded_docs = map(encoder.encode, fin)

    level = "document"
    if args.split_sentences:
        level = "sentence"

    print(f"Vocab size: {tokenizer.vocab_size}")
    print(f"Output prefix: {args.output_prefix}")
    output_bin_files = {}
    output_idx_files = {}
    builders = {}
    for key in args.json_keys:
        output_bin_files[key] = "{}_{}_{}.bin".format(args.output_prefix,
                                                      key, level)
        output_idx_files[key] = "{}_{}_{}.idx".format(args.output_prefix,
                                                      key, level)
        builders[key] = indexed_dataset.make_builder(output_bin_files[key],
                                               impl=args.dataset_impl,
                                               vocab_size=tokenizer.vocab_size)

    startup_end = time.time()
    proc_start = time.time()
    total_bytes_processed = 0
    print("Time to startup:", startup_end - startup_start)

    for i, (doc, bytes_processed) in enumerate(encoded_docs, start=1):
        total_bytes_processed += bytes_processed
        for key, sentences in doc.items():
            if len(sentences) == 0:
                continue
            for sentence in sentences:
                builders[key].add_item(torch.IntTensor(sentence))
            builders[key].end_document()
        if i % args.log_interval == 0:
            current = time.time()
            elapsed = current - proc_start
            mbs = total_bytes_processed/elapsed/1024/1024
            print(f"Processed {i} documents",
                  f"({i/elapsed} docs/s, {mbs} MB/s).",
                  file=sys.stderr)

    for key in args.json_keys:
        builders[key].finalize(output_idx_files[key])

if __name__ == '__main__':
    main()

In [11]:
INPUT_JSON_FILE='../dataset/SV/webnyheter2013.json'
OUTPUT_PATH='../dataset/SV/customSentenceSplit'
VOCAB_FILE='../dataset/SV/32k/vocab.json'
MERGE_FILE='../dataset/SV/32k/merges.txt'
NUM_CPUS=16

---
## below is a ReRun cell overwrite MYpreprocess_data.py
Run the below MYpreprocess_data.py successfully to obtain CustomSentenceSplitter_text_document.idx and CustomSentenceSplitter_text_document.idx file
<a id="Rerun_Cell"></a>

Go to cell above to modify MYpreprocess_data.py <a href="./Day3-4_customize_process2mmap.ipynb#MODIFY_CELL">Jump to Modify MYpreprocess_data.py</a> 

In [ ]:
!python ./Megatron-LM/tools/MYpreprocess_data.py \
                       --input $INPUT_JSON_FILE \
                       --output-prefix $OUTPUT_PATH \
                       --json-keys text \
                       --vocab-file $VOCAB_FILE \
                       --merge-file $MERGE_FILE \
                       --dataset-impl mmap \
                       --tokenizer-type GPT2BPETokenizer \
                       --workers $NUM_CPUS \
                       --append-eod

In [12]:
!python ./Megatron-LM/tools/SOLUTION_preprocess_data.py \
                       --input $INPUT_JSON_FILE \
                       --output-prefix $OUTPUT_PATH \
                       --json-keys text \
                       --vocab-file $VOCAB_FILE \
                       --merge-file $MERGE_FILE \
                       --dataset-impl mmap \
                       --tokenizer-type GPT2BPETokenizer \
                       --workers $NUM_CPUS \
                       --append-eod

Opening ../dataset/SV/webnyheter2013.json
> building GPT2BPETokenizer tokenizer ...
 > padded vocab (size: 32000) with 0 dummy tokens (new size: 32000)
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
> building GPT2BPETokenizer tokenizer ...
Vocab size: 32000
Output prefix: ../dataset/SV/customSentenceSplit
Time to startup: 0.35861778259277344
 > padded vocab (size: 32000) with 0 dummy tokens (new size: 32000)
Proc

In [13]:
! ls ../dataset/SV/

32k				       webnyheter2013.json
56k				       webnyheter2013.txt
customSentenceSplit_text_document.bin  webnyheter2013_text_document.bin
customSentenceSplit_text_document.idx  webnyheter2013_text_document.idx
webbnyheter2013.xml


In [9]:
## clean up
!rm ./Megatron-LM/tools/MYpreprocess_data.py

rm: cannot remove './Megatron-LM/tools/MYpreprocess_data.py': No such file or directory


---
## Up Next : 

[The Challenge - Go Big or go home!](./Day3-5_run_Megatron_with_varying_config.ipynb)

## Back To Start Menu
[start menu](../Start_Here.ipynb)

-----


## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). 